In [4]:
pip install ZipFile

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ZipFile (from versions: none)
ERROR: No matching distribution found for ZipFile

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.content/kaggle.json

In [2]:
!kaggle datasets download -d kazanova/sentiment140

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done')

FileNotFoundError: [Errno 2] No such file or directory: '/content/sentiment140.zip'

In [7]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # Fixed the typo here
from sklearn.model_selection import train_test_split # Fixed another typo here
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')#stop word is not importance

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#DATA PROCESSING

In [9]:
#load data
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
#check the number of rows and coloumns
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#name the coloumn and reading dataset
coloumn_name=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=coloumn_name)

In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
#counting the number of missing value in tha datasets
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [14]:
#check how many postive and negative data
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [15]:
#change the label 4 to 1
twitter_data.replace({'target':{4:1}},inplace=True)

In [16]:
#streaming is procees of reducing a words to root word
port_stemmer=PorterStemmer()

In [17]:
def stemming(content):

  stemmed_content=re.sub('[^a-z A-Z]',' ', content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stemmer.stem(word) for word in stemmed_content if not word  in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [18]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [20]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [26]:
# separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [27]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [29]:
print(Y)

[0 0 0 ... 1 1 1]


In [30]:
#splitting data to training data and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)
#0.2 to split 20% data to test
#stratify is used to make equal distribution of target postive and ngative
#we can run without random but if we give random whenever you split it get same data

In [31]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [32]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [34]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [75]:
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

NotFittedError: The TF-IDF vectorizer is not fitted

In [67]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [47]:
print(X_test)

  (0, 53904)	0.23662705066222536
  (0, 41330)	0.3621075009813407
  (0, 154815)	0.3501587145032445
  (0, 5650)	0.17295837060294847
  (0, 150603)	0.2214371376881977
  (0, 11837)	0.16338171192787918
  (0, 67046)	0.28383380524479984
  (0, 163496)	0.180932348330391
  (0, 26316)	0.2701195948137176
  (0, 158685)	0.3174206399662331
  (0, 107797)	0.17983918796121573
  (0, 51603)	0.25774294336856257
  (0, 104606)	0.4459147041194029
  (1, 136317)	0.5125311002730158
  (1, 56881)	0.48516752160641513
  (1, 83954)	0.35024490258566765
  (1, 134752)	0.4128221547743352
  (1, 141842)	0.21193862113578937
  (1, 99057)	0.2461083000957446
  (1, 2443)	0.2650067441796073
  (1, 5650)	0.181974929607951
  (2, 34981)	0.37310298419995075
  (2, 155497)	0.4431218247230756
  (2, 8454)	0.35890991973427855
  (2, 13273)	0.3887964554215523
  :	:
  (319994, 157662)	0.1692823231111289
  (319995, 114520)	0.4048122176535305
  (319995, 42067)	0.33609234404387445
  (319995, 173051)	0.3220764690826452
  (319995, 42660)	0.2995385

In [48]:
#Training the Machine Learning Model using Logistic Regression it is classification model
model=LogisticRegression(max_iter=1000)

In [49]:
model.fit(X_train,Y_train)
#here y train contain it's posititve or negative(0-neg,1-postive,it helps to figure out the words which is positive and negative)

LogisticRegression(max_iter=1000)

In [50]:
#Model evaluation

LogisticRegression(max_iter=1000)


In [51]:
#accurancy scores on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [52]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.81018984375


In [56]:
#accurancy sconres on the training data
#X_test_prediction=model.predict(X_test)
#test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [57]:
#saving trained model
import pickle

In [59]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))#wb is wrinting

In [60]:
#load to use it for future predictions
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [70]:
# Assuming 'vectorizer' is the variable name of the fitted vectorizer used during training
X_new = X_test[200]
print(X_new)
print(Y_test[200])
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Negative')
else:
  print('The news is Positive')

  (0, 49526)	0.3304618761547877
  (0, 155276)	0.3329216253277204
  (0, 165130)	0.4945833531038195
  (0, 126617)	0.30379639031739375
  (0, 150247)	0.46271222437867615
  (0, 144437)	0.34429228768787734
  (0, 93466)	0.2010391018276168
  (0, 57097)	0.18626501393086348
  (0, 36598)	0.18788837651908366
1


ValueError: X has 179187 features, but LogisticRegression is expecting 461488 features as input.